In [1]:
%load_ext autoreload
%autoreload 2
import openai
from langchain.llms import OpenAI
import random
from matplotlib import pyplot as plt 
import regex as re
import pandas as pd
from langdetect import detect
import seaborn as sns
import matplotlib.pyplot as plt
from sampling_methods import * 


import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from langchain.llms import OpenAI
from langdetect import detect_langs
from langchain.agents import load_tools
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType



# In Context Learning

In [98]:
openai.api_key=INSERT_API_KEY


def chat_with_model(prompt):
    content_var = "Your task is to discuss political election questions and provide persuasive answers with supporting reasons. Please engage in a thoughtful and informed discussion, presenting well-reasoned arguments to convince others of your perspective on political issues. Your responses should be clear, concise, and persuasive, addressing the concerns and questions raised about political elections. Consider supporting your arguments with relevant examples, statistics, or expert opinions to strengthen your persuasive approach.  Please note that your responses should be flexible enough to allow for various relevant and creative answers. You should focus on providing well-supported and convincing arguments to persuade others about the importance and significance of political elections in different contexts and scenarios."
    completion = openai.ChatCompletion.create(
        model='gpt-3.5-turbo-1106',
        messages=[{"role": "system", "content": content_var},{"role": "user", "content": prompt}],
        temperature = 0.1
    )
    return completion.choices[0].message["content"]

def prompting(tweets, topic, rating=True):
    question_var = f"""Do you agree with {topic}, with 1 indicating strong disagreement, 2 indicating weak disagreement, 3 indicating neutrality, 4 indicating weak agreement, and 5 indicating strong agreement? Tell us why did you feel that way. Please explain your choice in a sentence starts with your score and then your reasoning."""
    tweets_text = "\n".join([f"Your tweet: ### {i} ###" for i in tweets])
    prompt = f'''Here are your tweets, you should try to find patterns in them and be ready to answer other questions with same attitude.
{tweets_text}

Your question is:
{question_var}
'''
    return prompt


def prompting_candidate(tweets, rating=True):
    question_var = "Which candidate do you support? "
    tweets_text = "\n".join([f"Your tweet: ### {i} ###" for i in tweets])
    prompt = f'''Here are your tweets, you should try to find patterns in them and be ready to answer other questions with same attitude.
{tweets_text}

Your question is:
{question_var}
'''
    return prompt


# Read in data

In [3]:
biden = pd.read_csv('data/cleaned_tweets_biden.csv', lineterminator='\n')
trump = pd.read_csv('data/cleaned_tweets_trump.csv', lineterminator='\n')

llm = OpenAI(temperature=0,
             openai_api_key=INSERT_API_KEY
             )

# Sample from tweets

In [4]:
biden_users = sample_user(biden, 10)
trump_users = sample_user(trump, 10)
biden_tweets = sample_tweet_group(biden, biden_users).assign(hashtag='biden')
trump_tweets = sample_tweet_group(trump, trump_users).assign(hashtag='trump')
df = pd.concat([biden_tweets, trump_tweets])
df

,user_id,cleaned_tweets,hashtag
208075,1.864362e+09,TrumpNowMoreThanEver2020 JoeBidenToCreepyToBe...,biden
198817,1.864362e+09,chrisacorbett cenkuygur JoeBiden TrumpNowMo...,biden
82795,1.864362e+09,Trump StopChildTrafficking JoeBidenToCreepyT...,biden
171618,1.864362e+09,Trump StopChildTrafficking JoeBidenToCreepyT...,biden
207750,1.864362e+09,"Trump, on 'Fox &amp; Friends,' claims 'solid c...",biden
...,...,...,...
66628,8.186762e+17,"THIS 👉 GOP. HispanicCaucus, TruthMatters ...",trump
1689,8.186762e+17,JoeBiden Joe2020 BidenForPresident Vote For ...,trump
66198,8.186762e+17,JoeBiden Joe2020 BidenForPresident Rise Up |...,trump
105773,8.186762e+17,JoeBiden Joe2020 JoeBidenForPresident Calls ...,trump


# Which candidate each agent supports

In [99]:
def run():
    df_result = pd.DataFrame({'user_id':[], 'rating':[], 'reason':[]})

    for u in df['user_id'].unique():
        tweets = df[df['user_id']==u]['cleaned_tweets']
        prompt = prompting_candidate(tweets)
        result = chat_with_model(prompt)
        rating = result.split("\n")[0]
        reason = "\n".join(result.split("\n")[1:])
        df_result.loc[len(df_result)] = ({'user_id':u, 
                                          'topic':topic, 
                                          'rating':rating, 
                                          'reason':reason})
    return df_result

run = run()

In [100]:
support_list = []
for i in run["rating"]:
    print(i)
    if "Biden" in i:
        support_list.append(1)
    elif "Trump" in i:
        support_list.append(-1)
    elif "Neutral" in i:
        support_list.append(0)
    else:
        support_list.append(0)

It seems like you are a strong supporter of President Trump and his re-election campaign. Your tweets consistently express your support for Trump and your opposition to Joe Biden. It's clear that you are passionate about your political beliefs and are actively advocating for Trump's re-election.
It seems like the tweets provided are not directly related to expressing support for a specific candidate. However, I can assure you that I am committed to engaging in a thoughtful and informed discussion about political elections and providing well-reasoned arguments to address concerns and questions related to political issues. If you have specific questions or concerns about political candidates or their policies, I would be more than happy to discuss and provide persuasive answers with supporting reasons.
I understand that my previous tweets may give the impression of supporting Trump, but I want to clarify that my tweets reflect a nuanced and complex perspective on the political landscape.

In [101]:
support_list

[1, 0, -1, 1, -1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, -1, 0]

# Methods to extract agent perspectives

In [5]:
def run(topic):
    df_result = pd.DataFrame({'user_id':[], 'topic':[], 'rating':[], 'reason':[]})

    for u in df['user_id'].unique():
        tweets = df[df['user_id']==u]['cleaned_tweets']
        prompt = prompting(tweets, topic)
        result = chat_with_model(prompt)
        rating = result.split("\n")[0]
        reason = "\n".join(result.split("\n")[1:])
        df_result.loc[len(df_result)] = ({'user_id':u, 
                                          'topic':topic, 
                                          'rating':rating, 
                                          'reason':reason})
        
    return df_result

In [6]:
def format_final_result(topic):
    df_result = run(topic)
    each_topic = []
    for i in range(len(df_result["rating"])):
        print("Agent " + str(i))
        value = df_result["rating"][i]
        
        #extracting the score
        try:
            try:
                score = int(value[0])
            except:
                if "Score" in value:
                    match = (re.search(r"Score:\s*(\S+)", value))
                    score = int(match.group(1))
        except:
            score = 3
            
        each_topic.append(score)
        print(value)
        print()
    final_results.append(each_topic)

# Store and print all agent perspectives

In [7]:
final_results = []

In [8]:
topic = "The US needs to focus on improving LGBTQ+ rights."
format_final_result(topic)

Agent 0
3 I believe that the US should focus on improving LGBTQ+ rights. It's important to ensure equal rights and protections for all individuals, regardless of their sexual orientation or gender identity. LGBTQ+ individuals deserve the same rights and opportunities as everyone else, and it's crucial for the US to work towards achieving equality and inclusivity for all.

Agent 1
5: I strongly agree that the US needs to focus on improving LGBTQ+ rights. LGBTQ+ individuals deserve equal rights and protections under the law, including marriage equality, anti-discrimination laws, and access to healthcare. It's essential to create a more inclusive and equitable society for all Americans, regardless of their sexual orientation or gender identity.

Agent 2
4: I strongly agree that the US needs to focus on improving LGBTQ+ rights. The LGBTQ+ community deserves equal rights and protections under the law, and it is essential for the US to prioritize and advance these rights to create a more inc

In [9]:
topic = "The US needs to increase gun control policies."
format_final_result(topic)

Agent 0
3: Neutrality. Gun control is a complex issue with valid arguments on both sides. While it's important to ensure public safety, it's also crucial to protect Second Amendment rights. Striking a balance that addresses both concerns is essential.

Agent 1
4: I strongly agree that the US needs to increase gun control policies. The prevalence of gun violence in the US is a significant public health and safety concern, and stricter gun control measures can help reduce the frequency and severity of such incidents. Implementing comprehensive background checks, restricting access to high-capacity magazines and military-style assault weapons, and promoting responsible gun ownership are essential steps to address this pressing issue and protect the well-being of American citizens.

Agent 2
4 I strongly agree that the US needs to increase gun control policies. The prevalence of gun violence in the US is a significant issue that needs to be addressed through stricter gun control measures. I

In [10]:
topic = "Abortion should be legalized in the US."
format_final_result(topic)

Agent 0
3. I chose neutrality (3) because the issue of abortion is complex and deeply personal, involving considerations of individual rights, medical ethics, and religious beliefs. It's important to respect diverse perspectives and engage in thoughtful, empathetic discussions to find common ground and solutions that prioritize the well-being of women and families.

Agent 1
5: I strongly agree that abortion should be legalized in the US. The decision to have an abortion is a deeply personal one that should be made by a woman in consultation with her healthcare provider, without government interference. Legalizing abortion ensures women have access to safe and legal reproductive healthcare, protecting their autonomy and bodily integrity.

Agent 2
4: I strongly agree that abortion should be legalized in the US. 

Agent 3
3. I feel neutral about the legalization of abortion in the US. Abortion is a complex and deeply personal issue, and I believe it should be a decision made by individual

In [11]:
topic = "Universities in the US should practice affirmative action."
format_final_result(topic)

Agent 0
3 I believe that universities in the US should practice affirmative action to promote diversity and provide opportunities to underrepresented groups, but it should be implemented in a fair and balanced manner to ensure equal access to education for all individuals.

Agent 1
4. I strongly agree that universities in the US should practice affirmative action. Affirmative action is essential for promoting diversity and providing opportunities for underrepresented groups, which enriches the educational experience for all students and prepares them for a diverse workforce.

Agent 2
4: I strongly agree that universities in the US should practice affirmative action. Affirmative action is essential for promoting diversity and providing equal opportunities for underrepresented groups in higher education. It helps address historical and systemic inequalities, enriches the learning environment, and prepares students for a diverse workforce and society.

Agent 3
4. I strongly agree that uni

In [12]:
topic = "Climate change in the US is a cause for concern."
format_final_result(topic)

Agent 0
3: Neutrality. Climate change is a complex issue that requires careful consideration of scientific evidence and policy implications. While there is evidence of climate change and its potential impact, the extent of human contribution and the most effective solutions are still debated.

Agent 1
5: I strongly agree that climate change in the US is a cause for concern. The increasing frequency and intensity of natural disasters, such as hurricanes, wildfires, and extreme weather events, pose significant threats to the environment, public health, and the economy. The scientific consensus on human-induced climate change and its potential catastrophic impacts necessitate urgent action to mitigate its effects and transition to sustainable energy sources.

Agent 2
4: I strongly agree that climate change in the US is a cause for concern. The increasing frequency and intensity of natural disasters, such as hurricanes and wildfires, are clear indicators of the impact of climate change. Th

In [13]:
topic = "The US needs more paths for citizenship for undocumented immigrants."
format_final_result(topic)

Agent 0
3: I feel neutral about this issue. The US immigration system is complex, and there are valid arguments on both sides. On one hand, providing more paths for citizenship could address the reality of undocumented immigrants already contributing to the economy and society. On the other hand, there are concerns about the rule of law and the potential impact on legal immigration processes.

Agent 1
5: I strongly agree that the US needs more paths for citizenship for undocumented immigrants. The current immigration system is broken and inhumane, leaving millions of people living in the shadows without a clear path to legal status. Providing more paths for citizenship not only aligns with American values of fairness and compassion but also benefits the economy and society as a whole by integrating hardworking individuals into the fabric of the nation.

Agent 2
4. I strongly agree that the US needs more paths for citizenship for undocumented immigrants. 

Agent 3
4. I strongly agree th

In [14]:
topic = "The death penalty should be banned in the US."
format_final_result(topic)

Agent 0
3: I feel neutral about the death penalty. While I understand the arguments for its deterrent effect and justice for heinous crimes, I also recognize the potential for wrongful convictions and the irreversible nature of the punishment.

Agent 1
5: I strongly agree that the death penalty should be banned in the US. The justice system is fallible, and the irreversible nature of the death penalty means that innocent individuals could be wrongfully executed. Additionally, there is evidence of racial and socioeconomic disparities in the application of the death penalty, raising serious concerns about fairness and justice.

Agent 2
4. I strongly agree that the death penalty should be banned in the US. The death penalty is irreversible and has been shown to be applied unfairly, disproportionately affecting marginalized communities. Additionally, there have been cases of innocent individuals being wrongfully convicted and sentenced to death, highlighting the inherent flaws in the syste

In [15]:
topic = "Women in the US military should be allowed to serve in combat roles."
format_final_result(topic)

Agent 0
3 I believe that women in the US military should be allowed to serve in combat roles. While physical differences between men and women may exist, many women have demonstrated the capability to excel in combat positions, and their inclusion can enhance the overall effectiveness and diversity of the military.

Agent 1
5. I strongly agree that women in the US military should be allowed to serve in combat roles. Women have demonstrated their capability and dedication in various military positions, and denying them the opportunity to serve in combat roles based on their gender is unjust and limits the effectiveness of the military. Allowing women to serve in combat roles enhances the diversity and strength of the armed forces, contributing to a more inclusive and capable military.

Agent 2
5: I strongly agree that women in the US military should be allowed to serve in combat roles. Women have demonstrated their capability and dedication in various military positions, and denying the

In [16]:
topic = "Marijuana should be legalized in the US."
format_final_result(topic)

Agent 0
3. I feel neutral about the legalization of marijuana in the US. While I understand the potential benefits of legalization, such as tax revenue and reduced strain on the criminal justice system, I also have concerns about the potential impact on public health and safety. I believe that further research and careful consideration are necessary before making a definitive decision on this issue.

Agent 1
4: I strongly agree that marijuana should be legalized in the US. The criminalization of marijuana has disproportionately affected communities of color, leading to unjust incarceration rates and perpetuating systemic inequalities. Legalization would allow for regulation, taxation, and the redirection of law enforcement resources toward more pressing issues, promoting social justice and economic growth.

Agent 2
5: I strongly agree that marijuana should be legalized in the US. The criminalization of marijuana has disproportionately affected communities of color, leading to mass inca

In [17]:
topic = "Student loan interest rates should be reduced."
format_final_result(topic)

Agent 0
3 I believe that reducing student loan interest rates can be beneficial for students and the economy as a whole. Lower interest rates can make education more accessible and affordable, potentially reducing the burden of student debt and allowing individuals to invest in other areas of the economy, ultimately contributing to economic growth.

Agent 1
5. I strongly agree that student loan interest rates should be reduced. The burden of high-interest rates disproportionately affects students and graduates, hindering their ability to invest in their future and contribute to the economy. Lowering interest rates would alleviate financial stress and promote economic growth.

Agent 2
4 I strongly agree that student loan interest rates should be reduced. The burden of student loan debt is a significant barrier for many individuals seeking higher education and can have long-term negative effects on their financial well-being. Reducing interest rates would make education more accessible a

# Organize all agent perspectives for visualization data

In [18]:
df_final_results = pd.DataFrame(final_results).transpose()

titles = ['1-lgbtq', "2-guns","3-abortion", "4-affirmative_action", 
                   "5-climate_change", "6-citizenship","7-death_penalty", "8-women_in_military",
                   "9-marijuana", "10-student_loans"]
df_final_results.columns = titles
df_final_results

,1-lgbtq,2-guns,3-abortion,4-affirmative_action,5-climate_change,6-citizenship,7-death_penalty,8-women_in_military,9-marijuana,10-student_loans
0,3,3,3,3,3,3,3,3,3,3
1,5,4,5,4,5,5,5,5,4,5
2,4,4,4,4,4,4,4,5,5,4
3,5,3,3,4,5,4,5,5,4,5
4,3,3,3,4,4,3,3,3,4,3
5,4,3,3,3,3,3,3,4,3,3
6,4,4,3,4,5,3,4,5,3,4
7,4,5,5,4,5,4,5,5,4,4
8,3,3,3,3,3,3,3,3,3,3
9,4,4,4,4,4,4,4,4,4,4


In [19]:
df_final_results.to_csv("data/response_data_for_visualizations.csv", index = False)

In [104]:
df_with_support = df_final_results.copy()
df_with_support["biden_or_trump"] = support_list
df_with_support.to_csv("data/response_data_with_support_for_visualizations.csv", index = False)